<a href="https://colab.research.google.com/github/MaggieHDez/ClassFiles/blob/main/practica2_MCHD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Extracción de Información con Expresiones Regulares**

>**Alumno:** Margarita Cristina Hernández Delgadillo\
>**Matrícula:** 255879

En este cuaderno, analizaremos información clave del archivo `informe_acciones.txt`
utilizando expresiones regulares en Python. Extraeremos datos como el nombre de las empresas, fechas del valor de la acción y valores de acciones para calcular estadísticas como el valor máximo, mínimo y
promedio para cada empresa

## Conexión a folder contenedor del archivo

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Apertura de archivo

Corre este codigo para seleccionar el archivo ```informe_acciones.txt``` para cargarlo

In [ ]:
from google.colab import files
import re
uploaded = files.upload()
filename = list(uploaded.keys())[0]  # Extrae el nombre del primer archivo cargado
filename = re.sub(r'\s*\(.*\)', '', filename)

# Mostrar el nombre del archivo
print(f"El archivo cargado es: {filename}")

Saving informe_acciones.txt to informe_acciones (4).txt
El archivo cargado es: informe_acciones.txt


Después de cargar el archivo, puedes leer su contenido corriendo el siguiente código.

### Problema de lectura

Al realizar la lectura del archivo, me encontre con un problema ya que el archivo no era codificación utf. Para resolver esto utilicé chardet para detectar la codificación y poder realizar la lectura del archivo.

In [40]:
import chardet

with open(filename, 'rb') as f:
    result = chardet.detect(f.read())

### Extracción de datos

Una vez resulto el problema de codificación, procedemos a extraer los datos: Nombre de empresa, fechas y valor.

In [41]:
# Diccionario que guardará resultados
acciones = {}

# Abre el archivo con la codificación correcta
with open(filename, encoding='ISO-8859-1') as arch:
  for linea in arch:
    linea = linea.strip()
    # Extracción de nombre de empresa
    match = re.findall(r"acción de (.+?) \(",linea)
    if match:
      empresa = match[0]
      if empresa not in acciones:
        acciones[empresa] = {"fecha": [], "valores": []} # Inicialización

    match = re.findall(r"(\d{4}-\d{2}-\d{2})", linea)
    if match:
      fecha = match[0]
      acciones[empresa]["fecha"].append(fecha)

    match = re.findall(r"\$(\d+\.\d+)", linea)
    if match:
      valor = float(match[0])
      acciones[empresa]["valores"].append(valor)

### Cálculo de mínimo, máximo y promedio
Finalizamos con el cálculo de valor máximo, valor mínimo y el promedio y mostramos la información de la empresa en un tabla. Para esto utilizamos los data frames de ```pandas``` y ```max()```, ```min()``` y ```mean()``` de ```numpy```.

In [42]:
import pandas as pd
import numpy as np

# Calcular min, max y promedio
resultados = []
for empresa, datos in acciones.items():
  if datos["valores"]:  # Asegurar que hay datos para calcular
    max_valor = np.max(datos["valores"])
    min_valor = np.min(datos["valores"])
    promedio = np.mean(datos["valores"])
    fecha_reciente = max(datos["fecha"])  # Última fecha registrada

    resultados.append([empresa, fecha_reciente, max_valor, min_valor, promedio])

# Convertir resultados a DataFrame y mostrar como tabla
df_resultados = pd.DataFrame(resultados, columns=["Empresa", "Fecha más reciente", "Valor Máx ($)", "Valor Mín ($)", "Promedio ($)"])
df_sort = df_resultados.sort_values(by='Empresa',ascending=True) # Ordenamos por nombre de empresa
print(df_sort.to_string(index=False))  # Para imprimir toda la tabla de una vez sin los índices. Forzamos la impresión completa de la tabla

                        Empresa Fecha más reciente  Valor Máx ($)  Valor Mín ($)  Promedio ($)
                      AT&T Inc.         2024-09-02        1499.95         101.37    777.280093
                    AbbVie Inc.         2024-09-02        1493.72         104.19    811.056505
                     Adobe Inc.         2024-09-02        1499.69         105.72    803.033778
                  Alphabet Inc.         2024-09-02        1497.57         101.77    783.619501
                Amazon.com Inc.         2024-09-02        1498.27         103.83    762.746842
                     Apple Inc.         2024-09-02        1499.82         101.29    815.535308
        Berkshire Hathaway Inc.         2024-09-02        1497.75         102.99    780.418283
             Cisco Systems Inc.         2024-09-02        1499.73         104.72    794.882188
        Exxon Mobil Corporation         2024-09-02        1498.51         104.62    822.573687
              Intel Corporation         2024-09-02